In [1]:
import pandas as pd
import numpy as np

# Data Reading

In [2]:
Doc = pd.read_csv("archive/all-data.csv",header=None)
Doc
lines = Doc[1]
lines

0       According to Gran , the company has no plans t...
1       Technopolis plans to develop in stages an area...
2       The international electronic industry company ...
3       With the new production plant the company woul...
4       According to the company 's updated strategy f...
                              ...                        
4841    LONDON MarketWatch -- Share prices ended lower...
4842    Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843    Operating profit fell to EUR 35.4 mn from EUR ...
4844    Net sales of the Paper segment decreased to EU...
4845    Sales in Finland decreased by 10.5 % in Januar...
Name: 1, Length: 4846, dtype: object

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(lines)
analyze = vectorizer.build_analyzer()
#print(len(vectorizer.get_feature_names()))
X.toarray()

array([[0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.147059, 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       ...,
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ]])

In [4]:
final = pd.DataFrame(X.todense(),columns = vectorizer.get_feature_names())
final

,00,000,000063,002,0025,003,0030,007,008,01,...,zero,zgody,zinc,zip,zloty,zoltan,zone,zoo,zte,zu
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.147059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4841,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4842,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4843,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4844,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
final.sum()

00         4.970592
000       36.129250
000063     0.279671
002        0.359944
0025       0.232234
            ...    
zoltan     0.217377
zone       1.767678
zoo        0.233001
zte        0.279671
zu         0.207245
Length: 10064, dtype: float64

# Data Cleaning

In [6]:
final = final.drop(final.columns[final.sum() < 0.5], axis=1)
#final['Sentiment'] = Doc[0]

In [7]:
all_inputs = final[final.columns].values

In [8]:
all_labels = Doc[0].values

# Model application

## Model 1 - Decision Tree Classifier

In [9]:
from sklearn.model_selection import train_test_split

(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(all_inputs, all_labels, test_size=0.2, random_state = True)

In [10]:
from sklearn import tree
classifier = tree.DecisionTreeClassifier()

In [11]:
classifier.fit(training_inputs, training_classes)
classifier.score(testing_inputs, testing_classes)

0.677319587628866

## Model 2 - Classification using Neural Networks

In [12]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [13]:
encoder = LabelEncoder()
encoder.fit(all_labels)
encoded_Y = encoder.transform(all_labels)

dummy_y = np_utils.to_categorical(encoded_Y)

In [14]:
model = Sequential()
model.add(Dense(64, input_dim=all_inputs.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(all_inputs,dummy_y, test_size=0.2,shuffle = True)

In [16]:
model.fit(X_train, y_train, epochs=30)

Epoch 1/30
122/122 [==============================] - 1s 2ms/step - loss: 0.9110 - accuracy: 0.5872
Epoch 2/30
122/122 [==============================] - 0s 3ms/step - loss: 0.6831 - accuracy: 0.6760
Epoch 3/30
122/122 [==============================] - 0s 3ms/step - loss: 0.4416 - accuracy: 0.8385
Epoch 4/30
122/122 [==============================] - 0s 3ms/step - loss: 0.2374 - accuracy: 0.9301
Epoch 5/30
122/122 [==============================] - 0s 2ms/step - loss: 0.1196 - accuracy: 0.9719
Epoch 6/30
122/122 [==============================] - 0s 2ms/step - loss: 0.0654 - accuracy: 0.9876
Epoch 7/30
122/122 [==============================] - 0s 3ms/step - loss: 0.0394 - accuracy: 0.9920
Epoch 8/30
122/122 [==============================] - 0s 3ms/step - loss: 0.0264 - accuracy: 0.9954
Epoch 9/30
122/122 [==============================] - 0s 3ms/step - loss: 0.0200 - accuracy: 0.9964
Epoch 10/30
122/122 [==============================] - 0s 3ms/step - loss: 0.0166 - accuracy: 0.9966

In [17]:
pred_train = model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {} \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {} \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))   

Accuracy on training data: 0.9994840025901794 
 Error on training data: 0.0005159974098205566
Accuracy on test data: 0.7360824942588806 
 Error on test data: 0.2639175057411194
